In [27]:
import pandas as pd
import yaml
import re
import numpy as np

In [11]:
with open("schedule_2017.yaml", "r") as sched_file:
    schedule = yaml.load(sched_file)
schedule

{'Illinois': ['Ball State',
  'Western Kentucky',
  '@South Florida',
  '',
  'Nebraska',
  '@Iowa',
  'Rutgers',
  '@Minnesota',
  'Wisconsin',
  '@Purdue',
  'Indiana',
  '@Ohio State',
  'Northwestern'],
 'Indiana': ['Ohio State',
  '@Virginia',
  '',
  'Georgia Southern',
  '@Penn State',
  'Charleston Southern',
  'Michigan',
  '@Michigan State',
  '@Maryland',
  'Wisconsin',
  '@Illinois',
  'Rutgers',
  '@Purdue'],
 'Iowa': ['Wyoming',
  '@Iowa State',
  'North Texas',
  'Penn State',
  '@Michigan State',
  'Illinois',
  '',
  '@Northwestern',
  'Minnesota',
  'Ohio State',
  '@Wisconsin',
  'Purdue',
  '@Nebraska'],
 'Maryland': ['@Texas',
  'Towson',
  '',
  'Central Florida(UCF)',
  '@Minnesota',
  '@Ohio State',
  'Northwestern',
  '@Wisconsin',
  'Indiana',
  'Rutgers',
  'Michigan',
  '@Michigan State',
  'Penn State'],
 'Michigan': ['Florida',
  'Cincinnati',
  'Air Force',
  '@Purdue',
  '',
  'Michigan State',
  '@Indiana',
  '@Penn State',
  'Rutgers',
  'Minnesota',
 

In [12]:
# Only two weeks remaining
schedule = {k: v[-2:] for k, v in schedule.items()}
schedule

{'Illinois': ['@Ohio State', 'Northwestern'],
 'Indiana': ['Rutgers', '@Purdue'],
 'Iowa': ['Purdue', '@Nebraska'],
 'Maryland': ['@Michigan State', 'Penn State'],
 'Michigan': ['@Wisconsin', 'Ohio State'],
 'Michigan State': ['Maryland', '@Rutgers'],
 'Minnesota': ['@Northwestern', 'Wisconsin'],
 'Nebraska': ['@Penn State', 'Iowa'],
 'Northwestern': ['Minnesota', '@Illinois'],
 'Ohio State': ['Illinois', '@Michigan'],
 'Penn State': ['Nebraska', '@Maryland'],
 'Purdue': ['@Iowa', 'Indiana'],
 'Rutgers': ['@Indiana', 'Michigan State'],
 'Wisconsin': ['Michigan', '@Minnesota']}

In [5]:
predictions = pd.read_csv("predictions.csv")
predictions

,Team,Current Wins,Games Remaining,AVG,Austin,Bill,Bret,Julia,Dale,David,...,Liz,Luke,Matt,Max,Neil,Phil C,Phil K,Ron,Sara,Sasha
0,Illinois,2,2,3.17,2,2,4,4,2,5,...,3,3,2,3,3,3,4,3,2,4
1,Indiana,4,2,5.35,5,5,6,6,3,6,...,6,6,5,6,6,4,6,6,4,6
2,Iowa,6,2,7.30,6,7,8,7,6,8,...,7,6,8,6,7,9,7,8,7,7
3,Maryland,4,2,4.35,3,6,5,6,3,5,...,3,4,5,3,3,3,4,6,4,3
4,Michigan,8,2,9.26,10,10,10,9,9,10,...,10,9,9,9,9,10,8,9,9,9
5,Michigan State,7,2,5.78,3,7,6,8,6,4,...,8,6,5,4,5,7,5,4,6,5
6,Minnesota,5,2,6.61,8,7,5,7,6,9,...,7,9,6,5,8,4,6,8,4,5
7,Nebraska,4,2,7.17,7,8,7,6,7,7,...,8,7,8,6,7,7,6,9,6,6
8,Northwester,7,2,7.30,9,8,9,7,7,8,...,7,9,8,8,8,7,8,6,5,9
9,Ohio State,8,2,10.87,11,11,11,11,10,11,...,10,12,10,11,11,12,11,11,10,11


In [7]:
# Subtract off current wins to know how far off we are right now
pickers = ["Austin", "Bill", "Bret", "Julia", "Dale", "David", "Dawn", "Diane", "Erica", "Glenn", "Jessica", "John", "Kim", "Liz", "Luke", "Matt", "Max", "Neil", "Phil C", "Phil K", "Ron", "Sara", "Sasha"]
for col in pickers:
    predictions[col] = predictions[col] - predictions['Current Wins']

In [19]:
predictions = predictions.melt(id_vars=["Team"], value_vars=pickers, var_name="picker", value_name="remaining_prediction")

In [15]:
# Build all remaining 14 games
games = []
for t1, rem in schedule.items():
    for t2 in rem:
        if t2[0] == '@': # WARNING: neutral sites will be problematic!
            game = (t2[1:], t1)
        else:
            game = (t1, t2)
        if game not in games:
            games.append(game)

assert len(games) == 14
games

[('Ohio State', 'Illinois'),
 ('Illinois', 'Northwestern'),
 ('Indiana', 'Rutgers'),
 ('Purdue', 'Indiana'),
 ('Iowa', 'Purdue'),
 ('Nebraska', 'Iowa'),
 ('Michigan State', 'Maryland'),
 ('Maryland', 'Penn State'),
 ('Wisconsin', 'Michigan'),
 ('Michigan', 'Ohio State'),
 ('Rutgers', 'Michigan State'),
 ('Northwestern', 'Minnesota'),
 ('Minnesota', 'Wisconsin'),
 ('Penn State', 'Nebraska')]

In [34]:
win_count = {p:0 for p in pickers}
for itr in range(pow(2, len(games))):
    pred = predictions.copy()
    for gn, game in enumerate(games):
        t = game[(itr >> gn) & 1]
        pred.loc[pred['Team'] == t, 'remaining_prediction'] -= 1
    outcome = pred.groupby('picker')['remaining_prediction'].aggregate(lambda x: np.sum(np.power(x, 2)))
    for picker in outcome[outcome == outcome.min()].index:
        win_count[picker] += 1

win_count

{'Austin': 0,
 'Bill': 40,
 'Bret': 0,
 'Dale': 1871,
 'David': 0,
 'Dawn': 0,
 'Diane': 0,
 'Erica': 7625,
 'Glenn': 0,
 'Jessica': 0,
 'John': 0,
 'Julia': 7810,
 'Kim': 0,
 'Liz': 437,
 'Luke': 0,
 'Matt': 228,
 'Max': 0,
 'Neil': 0,
 'Phil C': 0,
 'Phil K': 366,
 'Ron': 0,
 'Sara': 0,
 'Sasha': 0}

In [35]:
pow(2,14)

16384